In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import boxcox

In [2]:
from utils.lof import LOF
from utils.metrics import nan_euclidean, elastic_nan_euclidean
from utils import get_table_players

In [3]:
from utils import Dataset

In [4]:
euclidean_regularized = elastic_nan_euclidean(1, 0.3, 0.7)

In [5]:
_4dm4_impute = Dataset('4dm4.db')

In [6]:
_4dm4_data = _4dm4_impute.select('scores', columns=['player_name', 'round', 'beatmap_type', 'beatmap_tag', 'score_logit'], where={
    'beatmap_type': ['LN', 'RC', 'HB'],
})

In [7]:
_4dm4_table = get_table_players(_4dm4_data)

In [8]:
standardized_4dm = pd.DataFrame(index=_4dm4_table.index)

In [9]:
for col in _4dm4_table.columns:
    mu = np.nanmean(_4dm4_table[col]); sigma = np.nanstd(_4dm4_table[col], ddof=1)
    standardized_4dm[col] = _4dm4_table[col].apply(lambda x: (x - mu) / sigma)

In [10]:
standardized_4dm.head()

,Q_RC_1,Q_RC_2,Q_LN_1,Q_HB_1,RO32_RC_1,RO32_RC_4,RO32_RC_5,RO32_LN_3,RO32_HB_1,RO16_RC_1,...,GF_RC_7,GF_LN_2,GF_LN_3,GF_LN_4,GF_HB_1,GF_HB_2,GF_HB_3,SF_RC_2,SF_HB_3,GF_RC_3
drag_,-1.271311,-0.888601,-0.288915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JPBenedictGame,-0.133422,0.052810,-0.519245,-0.407806,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iVALORUS,-2.538866,-1.609407,-2.138151,-1.485972,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BL5,0.293309,-0.947536,0.162731,0.073857,-1.536132,-1.589838,-0.170574,-0.071868,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Tebo-,0.427527,-0.009594,0.551290,-0.779546,0.079211,1.000453,0.577630,0.090251,0.900541,0.542193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
lof_euclid = LOF(5, euclidean_regularized)

In [14]:
lof_values = lof_euclid.fit_predict(standardized_4dm.values)

In [15]:
lof_4dm = pd.DataFrame(index=standardized_4dm.index)

lof_4dm['lof'] = lof_values

In [16]:
lof_4dm.loc['shokoha']

lof    10.475996
Name: shokoha, dtype: float64

In [17]:
lof_4dm.sort_values(by='lof', ascending=False).head(16)

,lof
shokoha,10.475996
Ricizus,8.657503
[LS]Byte,6.332421
RaGe Radical,5.302446
Lazereed,4.891264
Gray Hardrock,4.357861
nanonbandusty,4.280493
diamondBIaze,3.943919
acolyde,3.922488
Micleak,3.909344
